In [3]:
%matplotlib inline
import os
import json
import pickle
import pyspark
from functools import reduce
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType
import pandas as pd
from collections import OrderedDict
from datetime import date
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline, PipelineModel

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer, IndexToString
from pyspark.ml import PipelineModel

In [4]:
STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
STAGING_STORAGE_ACCOUNT_NAME = os.getenv('STAGING_STORAGE_ACCOUNT_NAME')
STAGING_STORAGE_ACCOUNT_KEY = os.getenv('STAGING_STORAGE_ACCOUNT_KEY')
AZUREML_NATIVE_SHARE_DIRECTORY = os.getenv('AZUREML_NATIVE_SHARE_DIRECTORY')

sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")

hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(STAGING_STORAGE_ACCOUNT_NAME), STAGING_STORAGE_ACCOUNT_KEY)

sql = SQLContext.getOrCreate(sc)

In [5]:
wasbUrlOutput = "wasb://{0}@{1}.blob.{2}/features.parquet".format(
            'intermediate',
            STAGING_STORAGE_ACCOUNT_NAME,
            STORAGE_ACCOUNT_SUFFIX)

dfa = sql.read.parquet(wasbUrlOutput)
dfa.printSchema()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- machineID: string (nullable = true)
 |-- ambient_temperature: double (nullable = true)
 |-- ambient_pressure: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- pressure: double (nullable = true)
 |-- f0: double (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- a0: double (nullable = true)
 |-- a1: double (nullable = true)
 |-- a2: double (nullable = true)
 |-- temperature_n: double (nullable = true)
 |-- pressure_n: double (nullable = true)
 |-- f0_n: double (nullable = true)
 |-- f1_n: double (nullable = true)
 |-- f2_n: double (nullable = true)
 |-- a0_n: double (nullable = true)
 |-- a1_n: double (nullable = true)
 |-- a2_n: double (nullable = true)



In [6]:
rdd = sc.sparkContext.parallelize([('MACHINE-008', 'F01'),('MACHINE-009', 'F02')])

failures_df = rdd.toDF(['machineID','failure'])
failures_df.printSchema()

root
 |-- machineID: string (nullable = true)
 |-- failure: string (nullable = true)



In [7]:
features = sorted([c for c in dfa.columns if c not in ['machineID', 'EnqueuedTimeUtc']])

# assemble features
va = VectorAssembler(inputCols=features, outputCol='features')

# this is a hack!! In the current simulated dataset, there are 4 machines: machine 0 is "good," 
# whereas the rest are experiencing different failures. This will eventually be produced by merging 
# telemetry with the "maintenance log"
feat_data = va.transform(dfa).join(failures_df, 'machineID', 'left_outer').fillna({'failure':'None'})

dfa.unpersist()
feat_data.persist(pyspark.StorageLevel.DISK_ONLY)
feat_data.limit(10).toPandas().head()

,machineID,EnqueuedTimeUtc,ambient_temperature,ambient_pressure,speed,temperature,pressure,f0,f1,f2,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,failure
0,MACHINE-009,2018-04-02T18:54:23.4630000Z,20.0,101.0,1102.0,49.789397,2203.998768,37.0,331.0,36.0,...,0.045181,1.999999,0.033575,0.300363,0.032668,9.880101,5.077440,3.583987,"[10887.8716529, 9.88010131843, 5595.33849704, ...",F02
1,MACHINE-009,2018-04-02T18:54:24.4600000Z,20.0,101.0,1102.0,50.455122,2203.999176,37.0,331.0,36.0,...,0.045785,1.999999,0.033575,0.300363,0.032668,9.852078,5.062152,3.612775,"[10856.9901607, 9.85207818578, 5578.49189018, ...",F02
2,MACHINE-009,2018-04-02T18:54:25.4760000Z,20.0,101.0,1102.0,51.113510,2203.999448,37.0,331.0,36.0,...,0.046382,1.999999,0.033575,0.300363,0.032668,9.880910,5.061872,3.583197,"[10888.7625725, 9.88090977539, 5578.18292251, ...",F02
3,MACHINE-009,2018-04-02T18:54:26.4760000Z,20.0,101.0,1102.0,51.764643,2203.999631,37.0,331.0,36.0,...,0.046973,2.000000,0.033575,0.300363,0.032668,9.885505,5.075601,3.579037,"[10893.8268045, 9.88550526722, 5593.31175837, ...",F02
4,MACHINE-009,2018-04-02T18:54:27.4770000Z,20.0,101.0,1102.0,52.408601,2203.999753,37.0,331.0,36.0,...,0.047558,2.000000,0.033575,0.300363,0.032668,9.858523,5.064370,3.606462,"[10864.0922766, 9.85852293706, 5580.93542663, ...",F02


In [8]:
# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(feat_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="failure", outputCol="indexedLabel").fit(feat_data)

deIndexer = IndexToString(inputCol = "prediction", outputCol = "predictedFailure", labels = labelIndexer.labels)

In [9]:
training, test = feat_data.randomSplit([0.8, 0.2], seed=12345)
print(training.count())
print(test.count())
feat_data.limit(5).toPandas().head()

35977
8877


,machineID,EnqueuedTimeUtc,ambient_temperature,ambient_pressure,speed,temperature,pressure,f0,f1,f2,...,temperature_n,pressure_n,f0_n,f1_n,f2_n,a0_n,a1_n,a2_n,features,failure
0,MACHINE-009,2018-04-02T18:54:23.4630000Z,20.0,101.0,1102.0,49.789397,2203.998768,37.0,331.0,36.0,...,0.045181,1.999999,0.033575,0.300363,0.032668,9.880101,5.077440,3.583987,"[10887.8716529, 9.88010131843, 5595.33849704, ...",F02
1,MACHINE-009,2018-04-02T18:54:24.4600000Z,20.0,101.0,1102.0,50.455122,2203.999176,37.0,331.0,36.0,...,0.045785,1.999999,0.033575,0.300363,0.032668,9.852078,5.062152,3.612775,"[10856.9901607, 9.85207818578, 5578.49189018, ...",F02
2,MACHINE-009,2018-04-02T18:54:25.4760000Z,20.0,101.0,1102.0,51.113510,2203.999448,37.0,331.0,36.0,...,0.046382,1.999999,0.033575,0.300363,0.032668,9.880910,5.061872,3.583197,"[10888.7625725, 9.88090977539, 5578.18292251, ...",F02
3,MACHINE-009,2018-04-02T18:54:26.4760000Z,20.0,101.0,1102.0,51.764643,2203.999631,37.0,331.0,36.0,...,0.046973,2.000000,0.033575,0.300363,0.032668,9.885505,5.075601,3.579037,"[10893.8268045, 9.88550526722, 5593.31175837, ...",F02
4,MACHINE-009,2018-04-02T18:54:27.4770000Z,20.0,101.0,1102.0,52.408601,2203.999753,37.0,331.0,36.0,...,0.047558,2.000000,0.033575,0.300363,0.032668,9.858523,5.064370,3.606462,"[10864.0922766, 9.85852293706, 5580.93542663, ...",F02


In [10]:
model_type = 'RandomForest' # Use 'DecisionTree', or 'RandomForest'

# train a model.
if model_type == 'DecisionTree':
    model = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                                      # Maximum depth of the tree. (>= 0) 
                                      # E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'
                                      maxDepth=15,
                                      # Max number of bins for discretizing continuous features. 
                                      # Must be >=2 and >= number of categories for any categorical feature.
                                      maxBins=32, 
                                      # Minimum number of instances each child must have after split. 
                                      # If a split causes the left or right child to have fewer than 
                                      # minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.
                                      minInstancesPerNode=1, 
                                      # Minimum information gain for a split to be considered at a tree node.
                                      minInfoGain=0.0, 
                                      # Criterion used for information gain calculation (case-insensitive). 
                                      # Supported options: entropy, gini')
                                      impurity="gini")

    ##=======================================================================================================================
    #elif model_type == 'GBTClassifier':
    #    cls_mthd = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
    ##=======================================================================================================================
else:    
    model = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                                      # Passed to DecisionTreeClassifier
                                      maxDepth=15, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0,
                                      impurity="gini",
                                      # Number of trees to train (>= 1)
                                      numTrees=50, 
                                      # The number of features to consider for splits at each tree node. 
                                      # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                                      featureSubsetStrategy="sqrt", 
                                      # Fraction of the training data used for learning each 
                                      # decision tree, in range (0, 1].' 
                                      subsamplingRate = 0.632)

# chain indexers and model in a Pipeline
pipeline_cls_mthd = Pipeline(stages=[labelIndexer, featureIndexer, model, deIndexer])

# train model.  This also runs the indexers.
model_pipeline = pipeline_cls_mthd.fit(training)

In [11]:
# make predictions. The Pipeline does all the same operations on the test data
predictions = model_pipeline.transform(test)
# predictions.limit(5).toPandas().head()
# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('failure', 'predictedFailure')
confuse = conf_table.toPandas().sort_values(by=['failure_predictedFailure'])
confuse.head()

,failure_predictedFailure,F01,F02,None
0,F01,880,0,0
2,F02,0,901,0
1,None,0,0,7096


In [12]:
model_path = os.path.join(AZUREML_NATIVE_SHARE_DIRECTORY, 'model')
model_archive_path = os.path.join(AZUREML_NATIVE_SHARE_DIRECTORY, 'model.tar.gz')

model_pipeline.write().overwrite().save(model_path)

import tarfile

tar = tarfile.open(model_archive_path, "w:gz")
tar.add(model_path, arcname="model")
tar.close()